In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
import time

Error: Session cannot generate requests

Load Dataset from Tensorflow Datasets and split into Train and Test

In [ ]:
train_ds, test_ds = tfds.load('fashion_mnist', split=['train', 'test'], as_supervised=True)

In [ ]:
BATCH_SIZE = 32
PREFETCH_SIZE = 64

Dataset Pipeline Function

In [ ]:
def prepare_fashion_dataset(data):
    
    data = data.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # normalize inputs from 0/255 to -1/1
    data = data.map(lambda img, target: ((img/128.)-1, target))
    # create one-hot vector for targets
    data = data.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
    # cache, shuffle, batch and prefetch data for efficient processing 
    data = data.cache()
    data = data.shuffle(1000).batch(BATCH_SIZE).prefetch(PREFETCH_SIZE)
    return data
    

In [ ]:
train_ds = prepare_fashion_dataset(train_ds)
test_ds =  prepare_fashion_dataset(test_ds)

# train_ds = train_ds.apply(prepare_fashion_dataset)
# test_ds =  test_ds.apply(prepare_fashion_dataset)

Construct a model:

In [ ]:
#custom model with custom layer
class CustomModel(tf.keras.Model):
    
    # Instantiate layer 
    def __init__(self):
        super(CustomModel, self).__init__()
        # Apply 24 kernel filters to our input
        # Convert from (28x28x1) to (28x28x24)
        self.conv1 = Conv2D(filters = 24, kernel_size = 3, padding = "same", activation = "relu")
        self.conv2 = Conv2D(filters = 24, kernel_size = 3, padding = "same", activation = "relu")
        # Cut the size by half and only keep most prominent values (reduce noise)
        # Convert (28x28x24) to (14x14x24)
        self.pooling = MaxPooling2D(pool_size = 2, strides = 2)
        # Apply 48 kernel filters
        # Convert from (14x14x24) to (14x14x48)
        self.conv3 = Conv2D(filters = 48, kernel_size = 3, padding = "same", activation = "relu")
        self.conv4 = Conv2D(filters = 48, kernel_size = 3, padding = "same", activation = "relu")
        # Take the mean of each of the 48 matrices
        # Convert output to (1x48)
        self.global_pool = GlobalAveragePooling2D()
        # Convert to (1x10) outputs
        self.out = Dense(10, activation = 'softmax')


    def fit(model):
        return model.fit()

    
    # cast the call-function as tf.function to increase efficiency
    @tf.function
    # pass the input through the layers of the network and return the output
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.pooling(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.global_pool(x)
        x = self.out(x)
        return x

Training Step:

In [ ]:
def train_step(model, input, target, loss_function, optimizer):
  # Keeps track of gradients
  with tf.GradientTape() as tape:
    prediction = model(input)
    loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
  
  # Reduce weights by gradients with selected optimizer
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

Testing Accuracy:

In [ ]:
def test(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (input, target) in test_data:
    # Calculate output and loss
    prediction = model(input)
    sample_test_loss = loss_function(target, prediction)
    # Check wether index of network output and the target for True is the same
    sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    # Take the mean over the whole dataset => accuracy of 1 epoch
    sample_test_accuracy = np.mean(sample_test_accuracy)
    # Keep track of accuracy and loss
    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = tf.reduce_mean(test_loss_aggregator)
  test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

  return test_loss, test_accuracy

In [ ]:
tf.keras.backend.clear_session()


In [ ]:
model = CustomModel()

model.fit()

Training:

In [ ]:
# Hyperparameters
num_epochs = 100
learning_rate = 0.05


# Instantiate the model
model = CustomModel()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer: SGD with default parameters. Check out 'tf.keras.optimizers'


optimizer = tf.keras.optimizers.SGD(learning_rate)

# Initialize lists for later visualization.
train_losses = []
test_losses = []
test_accuracies = []

#testing once before we begin
test_loss, test_accuracy = test(model, test_ds, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test(model, train_ds, cross_entropy_loss)
train_losses.append(train_loss)


# We train for num_epochs epochs.
for epoch in range(num_epochs):
    current_time = time.time()
    print(f'Epoch: {str(epoch)} starting with accuracy {round(float(test_accuracies[-1]), 2)}')

    #training (and checking in with training)
    epoch_loss_agg = []
    for input,target in train_ds:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)
    
    #track training loss
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    #testing, so we can track accuracy and test loss
    test_loss, test_accuracy = test(model, test_ds, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    print(f'Epoch took: {str(time.time()-current_time)}')

Error: Session cannot generate requests

In [ ]:
import matplotlib.pyplot as plt

# Visualize accuracy and loss for training and test data.
plt.figure()
line1, = plt.plot(train_losses)
line2, = plt.plot(test_losses)
line3, = plt.plot(test_accuracies)
plt.xlabel("Training steps")
plt.ylabel("Loss/Accuracy")
plt.legend((line1,line2, line3),("training","test", "test accuracy"))
plt.show()